<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/vaccine/%E7%97%87%E7%8A%B6%E5%88%A5%E5%A0%B1%E5%91%8A%E4%BB%B6%E6%95%B0%E9%9B%86%E8%A8%88%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.mhlw.go.jp/content/10906000/000784434.pdf -O data.pdf

--2021-05-29 01:14:36--  https://www.mhlw.go.jp/content/10906000/000784434.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 23.41.76.186, 2600:1417:9800:4a4::3194, 2600:1417:9800:4b3::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|23.41.76.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 663962 (648K) [application/pdf]
Saving to: ‘data.pdf’

data.pdf            100%[===================>] 648.40K  --.-KB/s    in 0.1s    

2021-05-29 01:14:36 (4.62 MB/s) - ‘data.pdf’ saved [663962/663962]



https://www.mhlw.go.jp/stf/newpage_18848.html

In [2]:
!apt install python3-tk ghostscript
!pip install camelot-py[cv]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts libcupsfilters1
  libcupsimage2 libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 11 newly installed, 0 to remove and 34 not upgraded.
Need to get 14.1 MB of archives.
After this operation, 

In [3]:
import camelot
import pandas as pd

In [4]:
tables = camelot.read_pdf("data.pdf", pages="8-18", flavor="stream")

In [5]:
df = pd.concat([table.df for table in tables]).iloc[1:].reset_index(drop=True).rename(columns={0: "症状", 1: "件数"})

In [6]:
df1 = df.copy()

In [7]:
df1["症状"] = df1["症状"].str.replace("\s", "", regex=True).str.normalize("NFKC")

df1["件数"] = pd.to_numeric(df1["件数"].str.replace(",", "")).astype("Int64")

df1["種類"] = df1["症状"].where(df1["件数"].isna()).fillna(method="ffill")

# 種類の行の箇所を削除
df1.dropna(subset=["件数"], inplace=True)

# 総計は種類を総計に変更
df1["種類"] = df1["種類"].mask(df1["症状"] == "総計", "総計")

# 一覧より集計

In [8]:
df2 = pd.read_csv("type.csv", names=["症状", "件数"])

In [9]:
df2["症状"] = df2["症状"].str.replace("\s", "", regex=True).str.normalize("NFKC")

In [10]:
# 総計を追加
df2.loc[df2.index[-1] + 1] = ["総計", df2["件数"].sum()]

In [11]:
df2

,症状,件数
0,発熱(発熱),2254
1,倦怠感(倦怠感),1494
2,頭痛(頭痛),1374
3,アナフィラキシー(アナフィラキシー反応),985
4,悪寒(悪寒),662
...,...,...
539,心室細動(心室細動),1
540,振動覚低下(振動覚低下),1
541,膿性痰(膿性痰),1
542,接触皮膚炎(接触皮膚炎),1


In [12]:
df3 = pd.merge(df1, df2, on="症状", how="outer")

In [13]:
df3.to_csv("lists.csv")

In [14]:
df3

,症状,件数_x,種類,件数_y
0,悪心・嘔吐(悪心),582,胃腸障害,582
1,胃食道逆流性疾患(胃食道逆流性疾患),2,胃腸障害,2
2,胃腸障害(胃腸障害),1,胃腸障害,1
3,胃潰瘍(胃潰瘍),1,胃腸障害,1
4,腹痛(下腹部痛),5,胃腸障害,5
...,...,...,...,...
539,脈圧低下(脈圧低下),1,臨床検査,1
540,脈拍異常(脈拍異常),1,臨床検査,1
541,脈拍異常(脈拍欠損),3,臨床検査,3
542,脈拍異常(橈骨動脈脈拍異常),2,臨床検査,2


In [15]:
df3[df3["件数_x"] != df3["件数_y"]]

,症状,件数_x,種類,件数_y
